In [39]:
# Imports
import os
from pathlib import Path
import pickle
import configparser
import pathlib
from direct_damages import damagescanner_rail_track as ds
import pandas as pd
import geopandas as gpd
from ci_adapt_utilities import *
import pickle
import shapely
import lonboard as lb
from tqdm.notebook import tqdm
from shapely import make_valid

In [2]:
# Load configuration with ini file (created running config.py)
config_file=r'C:\repos\ci_adapt\config_ci_adapt.ini'
config = configparser.ConfigParser()
config.read(config_file)

p = Path('..')
hazard_type = config.get('DEFAULT', 'hazard_type')
infra_type = config.get('DEFAULT', 'infra_type')
country_code = config.get('DEFAULT', 'country_code')
country_name = config.get('DEFAULT', 'country_name')
hazard_data_subfolders = config.get('DEFAULT', 'hazard_data_subfolders')
asset_data = config.get('DEFAULT', 'asset_data')
vulnerability_data = config.get('DEFAULT', 'vulnerability_data')
data_path = Path(pathlib.Path.home().parts[0]) / 'Data'
interim_data_path = data_path / 'interim' / 'collected_flood_runs'
interim_path = data_path / 'interim' / 'collected_flood_runs'

In [40]:
# Read exposure data (OSM, OpenStreetMap contributors (2024) / osm-flex)
assets_path = data_path / asset_data
assets=preprocess_assets(assets_path)
# assets['geometry'] = assets['geometry'].make_valid()
geom_dict = assets['geometry'].to_dict()
if 'collect_output' not in locals():
    collect_output_path = f'C:/Data/interim/collected_flood_runs/sample_collected_run.pkl'
    with open(collect_output_path, 'rb') as f:
        collect_output = pickle.load(f)

In [48]:
basin=2080447840
rp='M'
hazard_map='flood_DERP_RW_'+rp+'_4326_'+str(basin)

overlay_assets, hazard_numpified_list = load_baseline_run(hazard_map, interim_data_path, only_overlay=False)
collect_output_hm=collect_output[hazard_map]
collect_output_hm

{42: (760314.7387098153, 760314.7387098153),
 282: (908725.976584311, 1838967.5058219433),
 312: (0, 0),
 759: (42250.13609271769, 90860.4569941115),
 1346: (0, 0),
 1841: (0, 0),
 2044: (0, 0),
 4657: (0, 0),
 8077: (0, 0),
 10335: (65423.15581956348, 74842.5499504185),
 10338: (0, 0),
 16912: (2875845.999039334, 3836645.4630108397),
 17050: (0, 0),
 17954: (67562.70318656258, 76100.13053974957),
 17978: (0, 0),
 21849: (0, 0),
 21861: (0, 0),
 21995: (0, 0),
 21996: (0, 0),
 21997: (0, 0),
 21998: (0, 0),
 21999: (0, 0),
 22006: (0, 0),
 24260: (0.0, 402.03456092717823),
 24951: (0, 0),
 24952: (7475.248564856215, 24491.55079365034),
 25534: (0, 0),
 26022: (30501.318519391214, 48578.056788378715),
 26024: (33748.72816890179, 42278.34081988823),
 26059: (77536.47313638043, 249379.53247080083),
 26639: (3235.658499181394, 11883.137233300851),
 26642: (0.0, 138989.24380623378),
 26648: (0.0, 68944.07507000807),
 26651: (4550388.151588943, 5003826.561501219),
 26667: (0, 0),
 27481: (0,

In [49]:
import time

# find the exact hazard overlays:
hazard_numpified = hazard_numpified_list[1]
intersections = {}
timing = {'step1': 0, 'step2': 0, 'step3': 0, 'step4': 0}

for asset in tqdm(overlay_assets.groupby('asset'), total=len(overlay_assets.asset.unique())):
    start_time = time.time()    
    if collect_output_hm[asset[0]] == (0, 0):
        print(f'Asset {asset[0]} has no damages')
        continue

    asset_geom = geom_dict[asset[0]]
    get_hazard_points = hazard_numpified[asset[1]['hazard_point'].values]
    timing['step1'] += time.time() - start_time


    start_time = time.time()
    get_hazard_points[shapely.intersects(get_hazard_points[:, 1], asset_geom)]
    timing['step2'] += time.time() - start_time


    start_time = time.time()
    if len(get_hazard_points) == 0:  # no overlay of asset with hazard
        print(f'Damage for asset {asset[0]}: 0')
    else:
        if asset_geom.geom_type == 'LineString':
            intersection = shapely.intersection(get_hazard_points[:, 1], asset_geom)
            intersections[asset[0]] = get_hazard_points[:, 0], intersection
            overlay_meters = shapely.length(intersection)  # get the length of exposed meters per hazard cell
    timing['step3'] += time.time() - start_time


print('Timing summary:', timing)
    # break 


  0%|          | 0/68 [00:00<?, ?it/s]

Asset 312 has no damages
Asset 1346 has no damages
Asset 1841 has no damages
Asset 2044 has no damages
Asset 4657 has no damages
Asset 8077 has no damages
Asset 10338 has no damages
Asset 17050 has no damages
Asset 17978 has no damages
Asset 21849 has no damages
Asset 21861 has no damages
Asset 21995 has no damages
Asset 21996 has no damages
Asset 21997 has no damages
Asset 21998 has no damages
Asset 21999 has no damages
Asset 22006 has no damages
Asset 24951 has no damages
Asset 25534 has no damages
Asset 26667 has no damages
Asset 27481 has no damages
Asset 27690 has no damages
Asset 28374 has no damages
Asset 30448 has no damages
Asset 32480 has no damages
Asset 34468 has no damages
Asset 35256 has no damages
Asset 41397 has no damages
Asset 41398 has no damages
Asset 44216 has no damages
Asset 47207 has no damages
Asset 47209 has no damages
Timing summary: {'step1': 0.0019991397857666016, 'step2': 35.82637643814087, 'step3': 227.54518389701843, 'step4': 0}


In [53]:
intersections_df=pd.DataFrame(intersections, index=['hazard_value','geometry']).T
intersections_df

,hazard_value,geometry
42,[4.0],[LINESTRING (933945.9929397944 6312225.1336972...
282,"[1.0, 0.5, 2.0]",[MULTILINESTRING ((914004.2749710736 6271468.3...
759,"[4.0, 1.0, 2.0, 0.5]",[LINESTRING (904582.6615477379 6308841.5294425...
10335,"[4.0, 2.0]",[LINESTRING (867226.4780216607 6298694.9955237...
16912,"[2.0, 4.0, 4.0]",[MULTILINESTRING ((923469.4992591515 6284237.0...
17954,"[4.0, 1.0, 2.0]",[LINESTRING (904624.0513555499 6308850.2501437...
24260,[0.5],[LINESTRING (905480.8855055523 6296343.2372656...
24952,"[2.0, 1.0, 4.0]",[MULTILINESTRING ((930383.6022551736 6315765.8...
26022,"[4.0, 2.0, 0.5]",[LINESTRING (904606.1703412336 6308849.7933155...
26024,"[4.0, 2.0]",[LINESTRING (904611.5797639085 6308842.2683024...


In [54]:
#expand intersections df to have one row per hazard value and geometry
intersections_expanded_geom = intersections_df.explode('geometry').drop(columns='hazard_value')
intersections_expanded_hazard = intersections_df.explode('hazard_value').drop(columns='geometry')
intersections_expanded=gpd.GeoDataFrame(intersections_expanded_geom, geometry='geometry', crs='EPSG:3857')
intersections_expanded['hazard_value']=intersections_expanded_hazard['hazard_value'].values
intersections_expanded['osm_id']=assets.loc[intersections_expanded.index, 'osm_id'].values
intersections_expanded['length']=intersections_expanded['geometry'].length
intersections_expanded.sort_values('length', ascending=False)


,geometry,hazard_value,osm_id,length
34343,"MULTILINESTRING ((914624.697 6287304.032, 9146...",0.5,358229218,727.980601
26651,"LINESTRING (923553.980 6284346.021, 923639.567...",4.0,197654539,587.621747
282,"MULTILINESTRING ((914004.275 6271468.388, 9140...",1.0,4865585,529.056887
34343,"MULTILINESTRING ((915118.719 6286904.666, 9151...",1.0,358229218,378.795443
27924,"MULTILINESTRING ((913622.626 6271282.065, 9136...",2.0,212753319,348.182690
...,...,...,...,...
47117,"LINESTRING (868526.846 6294187.384, 868526.789...",1.0,1217532033,0.801636
24260,"LINESTRING (905480.886 6296343.237, 905480.650...",0.5,173762820,0.603922
24952,"LINESTRING (930384.920 6315763.610, 930385.204...",4.0,183890644,0.561434
30447,"LINESTRING (914652.477 6271782.822, 914652.856...",2.0,261924059,0.423370


In [63]:
intersections_expanded[intersections_expanded['osm_id']=='122391562'].length.sum()

556.2591752463261

In [55]:
m=lb.viz(intersections_expanded)
m

c:\Users\peregrin\AppData\Local\anaconda3\envs\ci_adapt_env\Lib\site-packages\lonboard\_geoarrow\ops\reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")


Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…